# Count variants in each sample
This Python Jupyter notebook counts occurrences of each barcode in each sample from Illumina barcode sequencing, and adds these counts to the codon variant table.

## Set up analysis
### Import Python modules.
Use [plotnine](https://plotnine.readthedocs.io/en/stable/) for ggplot2-like plotting.

The analysis relies heavily on the Bloom lab's [dms_variants](https://jbloomlab.github.io/dms_variants) package:

In [1]:
import itertools
import multiprocessing
import multiprocessing.pool
import os
import warnings

import alignparse
import alignparse.targets

import dms_variants.codonvarianttable
from dms_variants.constants import CBPALETTE
import dms_variants.illuminabarcodeparser
import dms_variants.utils
import dms_variants.plotnine_themes

from IPython.display import display, HTML

import pandas as pd

from plotnine import *

import yaml

Set [plotnine](https://plotnine.readthedocs.io/en/stable/) theme to the gray-grid one defined in `dms_variants`:

In [2]:
theme_set(dms_variants.plotnine_themes.theme_graygrid())

Versions of key software:

In [3]:
print(f"Using alignparse version {alignparse.__version__}")
print(f"Using dms_variants version {dms_variants.__version__}")

Using alignparse version 0.6.0
Using dms_variants version 1.4.3


Ignore warnings that clutter output:

In [4]:
warnings.simplefilter('ignore')

### Parameters for notebook
Read the configuration file:

In [5]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

Make output directory if needed:

In [6]:
os.makedirs(config['counts_dir'], exist_ok=True)
os.makedirs(config['figs_dir'], exist_ok=True)

## Input variant tables
Initialize the table of barcode-variant pairs from the respective `process_ccs` notebooks for each background.

In [7]:
variants = pd.read_csv(config['codon_variant_table_file_BA2'], na_filter=None)
variants = variants.append(pd.read_csv(config['codon_variant_table_file_BQ11'], na_filter=None))
variants = variants.append(pd.read_csv(config['codon_variant_table_file_XBB15'], na_filter=None))

variants = variants.reset_index(drop=True)

display(HTML(variants.head().to_html(index=False)))

target,library,barcode,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions
BA2,pool1,AAAAAAAAAAACGCGA,3,ATT88GTT,I88V,1,1
BA2,pool1,AAAAAAAAACAGCGAG,7,AGA163ATT,R163I,1,1
BA2,pool1,AAAAAAAAACCACGAA,5,CAA79ATG,Q79M,1,1
BA2,pool1,AAAAAAAAACCGAACT,3,GGT83AAA,G83K,1,1
BA2,pool1,AAAAAAAAACGCTATG,6,GAA135GAT,E135D,1,1


Are there any barcodes in the same library that are shared across targets?
If so, we need to get rid of those as they will be confounded in barcode parsing:

In [8]:
dup_barcodes = (
    variants
    .groupby(['library', 'barcode'])
    .size()
    .rename('duplicate_count')
    .reset_index()
    .query('duplicate_count > 1')
    )

print('Here are duplicated barcodes:')
display(HTML(dup_barcodes.head().to_html(index=False)))

print(f"\nRemoving the {len(dup_barcodes)} duplicated barcodes."
      f"Started with {len(variants)} barcodes:")
variants = (
    variants
    .merge(dup_barcodes, on=['library', 'barcode'], how='outer')
    .query('duplicate_count.isnull()', engine='python')
    )
print(f"After removing duplicates, there are {len(variants)} barcodes.")

Here are duplicated barcodes:


library,barcode,duplicate_count
pool1,AAAAAAAGTTATTTGG,2
pool1,AAAATAAAATTGTTTT,2
pool1,AACAACATATTATAAA,2
pool1,AAGTTGGTAAAAAAAA,2
pool1,AATTTAACTATAAAAT,2



Removing the 43 duplicated barcodes.Started with 649567 barcodes:
After removing duplicates, there are 649481 barcodes.


Pull out a target sequence for matching to the barcode and flanking sequence regions. Note, in this pipeline this is ok because our different backgrounds don't have differing flanks or other features within the actual N16 region covered in Illumina sequencing. If ever placing in-line barcodes here in the future, we would need to modify this.

In [9]:
# get wildtype gene sequence for primary target
targets = alignparse.targets.Targets(seqsfile=config['amplicons_BA2'],
                                     feature_parse_specs=config['feature_parse_specs_BA2'])

## Setup to parse barcodes
Read data frame with list of all barcode runs.

In [24]:
# barcode runs with R1 files by semicolon string split
barcode_runs = (pd.read_csv(config['barcode_runs'])
                .query('library=="pool2"')
                .query('sample=="TiteSeq_07_bin1"')
                .assign(R1=lambda x: x['R1'].str.split('; '))
                )

barcode_runs.loc[barcode_runs['library'] == 'pool2', 'library'] = 'pool1'

display(HTML(barcode_runs.to_html(index=False)))

library,sample,sample_type,sort_bin,concentration,date,number_cells,R1
pool1,TiteSeq_07_bin1,TiteSeq,1,7.0,230518,9527244,"[/uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2023/230628_Aerium-Vir-mAbs_XBB-polish/21073R/Fastq/21073X94_230712_A00421_0570_BHFTYKDRX3_S94_L001_R1_001.fastq.gz, /uufs/chpc.utah.edu/common/home/u6042467/starr-group1/sequencing/TNS/2023/230628_Aerium-Vir-mAbs_XBB-polish/21073R/Fastq/21073X94_230712_A00421_0570_BHFTYKDRX3_S94_L002_R1_001.fastq.gz]"


Make sure library / sample combinations are unique:

In [25]:
assert len(barcode_runs) == len(barcode_runs.groupby(['library', 'sample']))

Make sure the the libraries for which we have barcode runs are all in our variant table:

In [26]:
unknown_libs = set(barcode_runs['library']) - set(variants['library'])
if unknown_libs:
    raise ValueError(f"Libraries with barcode runs not in variant table: {unknown_libs}")

Now we initialize an [IlluminaBarcodeParser](https://jbloomlab.github.io/dms_variants/dms_variants.illuminabarcodeparser.html#dms_variants.illuminabarcodeparser.IlluminaBarcodeParser) for each library.

First, get the length of the barcode from the alignment target after making sure the same length for all targets:

In [27]:
bclen = len(targets.targets[0].get_feature('barcode').seq)

assert (bclen == len(target.get_feature('barcode').seq) for target in targets.targets)

print(f"Barcodes of length {bclen}")

Barcodes of length 16


The other barcode parsing params come from the config file:

In [28]:
parser_params = config['illumina_barcode_parser_params']

display(HTML(
    pd.Series(parser_params, name='value')
    .rename_axis(index='parameter')
    .reset_index()
    .to_html(index=False)
    ))

parameter,value
upstream,GGCCGC
downstream,
minq,20
upstream_mismatch,1
downstream_mismatch,0


The parser needs to know the set of valid barcodes, which are stored in the variant table and are different for each library.
So we create a different parser for each library using these valid barcode sets:

In [29]:
# create dict keyed by library, value is parser for library
parsers = {lib: dms_variants.illuminabarcodeparser.IlluminaBarcodeParser(
                    bclen=bclen,
                    valid_barcodes=variants.loc[variants['library']==lib]['barcode'],
                    **parser_params)
           for lib in set(variants['library'])}

print('Number of valid barcodes searched for by each parser:')
display(HTML(
    pd.DataFrame([(lib, len(p.valid_barcodes)) for lib, p in parsers.items()],
                 columns=['library', 'number of valid barcodes'])
    .to_html(index=False)
    ))

Number of valid barcodes searched for by each parser:


library,number of valid barcodes
pool1,314804
pool2,334677


## Parse barcodes
We now parse the barcodes.
Since this will take a while, we utilize multiple CPUs via the Python [multiprocessing](https://docs.python.org/3.6/library/multiprocessing.html) module.
First, determine how many CPUs to use.
We use the minimum of the user-specified number hardcoded below and the number actually available.
(If you are running *interactively* on the Hutch cluster, you may need to reduce the number below in order to avoid an error as there is an enforced CPU limit on the home `rhino` nodes):

In [30]:
ncpus = min(config['max_cpus'], multiprocessing.cpu_count())
print(f"Using {ncpus} CPUs")

Using 8 CPUs


Parse the barcodes in parallel via a [multiprocessing.Pool](https://docs.python.org/3.6/library/multiprocessing.html#multiprocessing.pool.Pool) using all the available CPUs to get a list of the data frames with barcode counts / fates for each sample:

In [31]:
def process_func(parser, r1files, library, sample):
    """Convenience function to be starmapped to multiprocessing pool."""
    return parser.parse(r1files, add_cols={'library': library, 'sample': sample})

# parallel computation of list of data frames
with multiprocessing.pool.Pool(processes=ncpus) as pool:
    bclist = pool.starmap(
                process_func,
                [(parsers[run.library], run.R1, run.library, run.sample)
                  for run in barcode_runs.itertuples()],
                )

Now concatenate the list into data frames of barcode counts and barcode fates:

In [41]:
counts = pd.concat([samplecounts for samplecounts, _ in bclist],
                   sort=False,
                   ignore_index=True)

print('First few lines of counts data frame:')
display(HTML(counts.query('count>100').head(20).to_html(index=False)))

fates = pd.concat([samplefates for _, samplefates in bclist],
                  sort=False,
                  ignore_index=True)

print('First few lines of fates data frame:')
display(HTML(fates.head().to_html(index=False)))

First few lines of counts data frame:


barcode,count,library,sample
CACCCACGCGAAACAT,4105,pool1,TiteSeq_07_bin1
AAGGAAAATATTAACT,3567,pool1,TiteSeq_07_bin1
TCCTGGACATGATGCC,3479,pool1,TiteSeq_07_bin1
TAGAAAGGATGAATAA,3061,pool1,TiteSeq_07_bin1
GCTTAAGAATTTCACA,2746,pool1,TiteSeq_07_bin1
AGCCTATGACCACGTT,2619,pool1,TiteSeq_07_bin1
AGTCGAGCTCACCTAT,2368,pool1,TiteSeq_07_bin1
CAACCTGGACCAATGC,2131,pool1,TiteSeq_07_bin1
CAAACGTACCATAGAA,2110,pool1,TiteSeq_07_bin1
ACTGCGACCCACGGCC,2073,pool1,TiteSeq_07_bin1


First few lines of fates data frame:


fate,count,library,sample
valid barcode,6818667,pool1,TiteSeq_07_bin1
invalid barcode,1196394,pool1,TiteSeq_07_bin1
low quality barcode,602714,pool1,TiteSeq_07_bin1
unparseable barcode,171070,pool1,TiteSeq_07_bin1
failed chastity filter,0,pool1,TiteSeq_07_bin1


## Examine fates of parsed barcodes
First, we'll analyze the "fates" of the parsed barcodes.
These fates represent what happened to each Illumina read we parsed:
 - Did the barcode read fail the Illumina chastity filter?
 - Was the barcode *unparseable* (i.e., the read didn't appear to be a valid barcode based on flanking regions)?
 - Was the barcode sequence too *low quality* based on the Illumina quality scores?
 - Was the barcode parseable but *invalid* (i.e., not in our list of variant-associated barcodes in the codon variant table)?
 - Was the barcode *valid*, and so will be added to variant counts.
 
First, we just write a CSV file with all the barcode fates:

In [33]:
fatesfile = os.path.join(config['counts_dir'], 'barcode_fates.csv')
print(f"Writing barcode fates to {fatesfile}")
fates.to_csv(fatesfile, index=False)

Writing barcode fates to results/counts/barcode_fates.csv


Next, we tabulate the barcode fates in wide format:

In [34]:
display(HTML(fates
             .pivot_table(columns='fate',
                          values='count',
                          index=['library', 'sample'])
             .to_html()
             ))

,fate,failed chastity filter,invalid barcode,low quality barcode,unparseable barcode,valid barcode
library,sample,,,,,
pool1,TiteSeq_07_bin1,0,1196394,602714,171070,6818667


Now we plot the barcode-read fates for each library / sample, showing the bars for valid barcodes in orange and the others in gray.
We see that the largest fraction of barcode reads correspond to valid barcodes, and most of the others are invalid barcodes (probably because the map to variants that aren't present in our variant table since we didn't associate all variants with barcodes). The exception to this is lib2 Titeseq_03_bin3; the PCR for this sample in the original sequencing run failed, so we followed it up with a single MiSeq lane. We did not filter out the PhiX reads from this data before parsing, so these PhiX reads will deflate the fraction of valid barcode reads as expected, but does not indicate any problems.

In [35]:
barcode_fate_plot = (
    ggplot(
        fates
        .assign(sample=lambda x: pd.Categorical(x['sample'],
                                                x['sample'].unique(),
                                                ordered=True),
                fate=lambda x: pd.Categorical(x['fate'],
                                              x['fate'].unique(),
                                              ordered=True),
                is_valid=lambda x: x['fate'] == 'valid barcode'
                ), 
        aes('fate', 'count', fill='is_valid')) +
    geom_bar(stat='identity') +
    facet_grid('sample ~ library') +
    facet_grid('sample ~ library') +
    scale_fill_manual(CBPALETTE, guide=False) +
    theme(figure_size=(1.4 * (1 + fates['library'].nunique()),
                       1.7 * (1.2 + fates['sample'].nunique())),
          axis_text_x=element_text(angle=90),
          panel_grid_major_x=element_blank()
          ) +
    scale_y_continuous(labels=dms_variants.utils.latex_sci_not,
                       name='number of reads')
    )

_ = barcode_fate_plot.draw()

## Output csv of barcode counts in variant-barcode lookup table

In [22]:
print(f"Writing variant counts to {config['variant_counts_file']}")
counts.to_csv(config['variant_counts_file'], index=False)

Writing variant counts to results/counts/variant_counts.csv


The [CodonVariantTable](https://jbloomlab.github.io/dms_variants/dms_variants.codonvarianttable.html#dms_variants.codonvarianttable.CodonVariantTable) has lots of nice functions that can be used to analyze the counts it contains.
However, we do that in the next notebook so we don't have to re-run this entire (rather computationally intensive) notebook every time we want to analyze a new aspect of the counts.